In [2]:
import os
import requests
import numpy as np

import torch
from torch import nn
import matplotlib.pyplot as plt

In [10]:
# download the tiny shakespeare dataset

input_file_path = os.path.join("data", "input.txt")
if not os.path.exists(input_file_path):
    data_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
    with open(input_file_path, "w", encoding="utf-8") as f:
        f.write(requests.get(data_url).text)

with open(input_file_path, "r", encoding="utf-8") as f:
    data_txt = f.read()
n = len(data)

data_txt[:61] # (long string)

'First Citizen:\nBefore we proceed any further, hear me speak.\n'

In [12]:
print(f"Number of characters: {len(data_txt)}")
print(data_txt[:250])

Number of characters: 1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [13]:
# list of possible chars and amount in dataset
chars = sorted(list(set(data_txt)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [8]:
# very simple encoder (chars to ints)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda e: "".join([itos[i] for i in e])
print(encode(" !$&',-.3:;?ABCDE"))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]


In [14]:
# encode entire dataset
data = torch.tensor(encode(data_txt), dtype=torch.long)
print(data.shape[0])
print(data[:250])
print(decode(data.numpy())[:250])

1115394
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57, 

In [24]:
# split into train and validation sets
n = int(0.9*len(data)) # 90%, 10%
train_data = data[:n]
val_data = data[n:]
print(f"len of train set: {len(train_data)}")
print(f"len of val set: {len(val_data)}")

len of train set: 1003854
len of val set: 111540


In [25]:
ctx_len = 8
print(train_data[:ctx_len+1])
decode(train_data.numpy()[:ctx_len+1])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])


'First Cit'

In [27]:
# this is what the context window looks like
x = train_data[:ctx_len]
y = train_data[1:ctx_len+1]
for t in range(ctx_len):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [47]:
# setup batch dimension
torch.manual_seed(42)
batch_size = 2 # usually 32
ctx_len = 8

def get_batch(split):
  data = train_data if split == "train" else val_data
  ix = torch.randint(len(data) - ctx_len, (batch_size,))
  x = torch.stack([data[i:i+ctx_len] for i in ix])
  y = torch.stack([data[i+1:i+ctx_len+1] for i in ix])
  return x, y

xb, yb = get_batch("train")
print(f"first input decoded: {decode(xb.numpy()[0])}")
print("inputs:")
print(xb.shape, xb)
print("targets:")
print(yb.shape, yb)
print("----")

for b in range (batch_size): # batch dimension
  for t in range(ctx_len): # time dimension
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when input is {context.tolist()} the target: {target}")

first input decoded: s his lo
inputs:
torch.Size([2, 8]) tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41]])
targets:
torch.Size([2, 8]) tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39]])
----
when input is [57] the target: 1
when input is [57, 1] the target: 46
when input is [57, 1, 46] the target: 47
when input is [57, 1, 46, 47] the target: 57
when input is [57, 1, 46, 47, 57] the target: 1
when input is [57, 1, 46, 47, 57, 1] the target: 50
when input is [57, 1, 46, 47, 57, 1, 50] the target: 53
when input is [57, 1, 46, 47, 57, 1, 50, 53] the target: 60
when input is [1] the target: 58
when input is [1, 58] the target: 46
when input is [1, 58, 46] the target: 43
when input is [1, 58, 46, 43] the target: 56
when input is [1, 58, 46, 43, 56] the target: 43
when input is [1, 58, 46, 43, 56, 43] the target: 1
when input is [1, 58, 46, 43, 56, 43, 1] the target: 41
when input is [1, 58, 46, 43, 56, 43, 1, 41] the target